In [2]:
from google.colab import files
import os

print("Click 'Choose Files' below and select your zip file...")
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    size_in_bytes = len(uploaded[filename])
    size_in_mb = size_in_bytes / (1024 * 1024)
    print(f'Uploaded: {filename} ({size_in_mb:.2f} MB)')

# ============================================
# YOLOv11 TRAINING - 100 EPOCHS
# ============================================

# 1. Install required packages FIRST (before importing ultralytics)
print("Installing packages...")
!pip install ultralytics pyyaml roboflow seaborn thop -q
print("✅ Packages installed\n")

# Now import the required packages
import zipfile
import yaml
from ultralytics import YOLO
import torch
import time

# 2. Check for existing zip file
print("Checking for dataset zip file...")
zip_files = [f for f in os.listdir('/content') if f.endswith('.zip')]

if not zip_files:
    print("No zip file found in /content. Please make sure you uploaded it correctly.")
    # Use the uploaded file from the initial upload
    zip_filename = list(uploaded.keys())[0]
else:
    # Use the first zip file found
    zip_filename = zip_files[0]
    print(f"Found zip file: {zip_filename}")

# 3. Extract dataset to a proper path in Colab
dataset_folder = "/content/datasets/traffic_dataset"
if os.path.exists(dataset_folder):
    !rm -rf "{dataset_folder}"

os.makedirs(dataset_folder, exist_ok=True)

print(f"Extracting {zip_filename} to {dataset_folder}...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

# 4. Find the actual data.yaml and fix paths
yaml_file = None
for root, dirs, files in os.walk(dataset_folder):
    if 'data.yaml' in files:
        yaml_file = os.path.join(root, 'data.yaml')
        dataset_root = root
        break

if yaml_file is None:
    raise FileNotFoundError("data.yaml not found in the extracted dataset")

print(f"Found data.yaml at: {yaml_file}")

# Read and update the data.yaml with correct Colab paths
with open(yaml_file, 'r') as f:
    data_config = yaml.safe_load(f)

# Update paths to absolute paths in Colab
data_config['train'] = os.path.join(dataset_root, 'train/images').replace('\\', '/')
data_config['val'] = os.path.join(dataset_root, 'valid/images').replace('\\', '/')
if 'test' in data_config and data_config['test']:
    data_config['test'] = os.path.join(dataset_root, 'test/images').replace('\\', '/')

# Also fix any path separators in the names
data_config['path'] = dataset_root.replace('\\', '/')

# Save the updated data.yaml
with open(yaml_file, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ Dataset prepared and data.yaml updated.")
print(f"Train path: {data_config['train']}")
print(f"Val path: {data_config['val']}")

# 5. Check GPU
print(f"\nGPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 6. Train the model
start_time = time.time()

# Load a model
model = YOLO('yolo11m.pt')  # Using medium model

# Train
results = model.train(
    data=yaml_file,
    epochs=100,
    imgsz=640,
    batch=16,
    patience=15,
    project='/content/runs',
    name='traffic_100epochs',
    exist_ok=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    verbose=True,
)

# Print training time
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/3600:.2f} hours.")

# 7. Evaluate the best model
best_model_path = '/content/runs/traffic_100epochs/weights/best.pt'
if os.path.exists(best_model_path):
    best_model = YOLO(best_model_path)
    metrics = best_model.val()

    # Print metrics
    print(f"\nValidation Results:")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall: {metrics.box.mr:.4f}")
    print(f"mAP@0.5: {metrics.box.map50:.4f}")
    print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")

    # Per-class metrics
    print("\nPer-class metrics:")
    for i, class_name in enumerate(metrics.names.values()):
        if i < len(metrics.box.p):
            print(f"  {class_name}:")
            print(f"    Precision: {metrics.box.p[i]:.4f}")
            print(f"    Recall: {metrics.box.r[i]:.4f}")
            print(f"    mAP50: {metrics.box.ap50[i]:.4f}")
else:
    print(f"Best model not found at {best_model_path}")

Click 'Choose Files' below and select your zip file...


Saving test102.v3i.yolov11.zip to test102.v3i.yolov11.zip
Uploaded: test102.v3i.yolov11.zip (0.90 MB)
Installing packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 140.3 MB/s eta 0:00:00
✅ Packages installed

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Checking for dataset zip file...
Found zip file: test102.v3i.yolov11.zip
E